Json import to openSearch

In [4]:
import createRecipesMap as crm

crm.CreateRecipesMap("../jsonData/recipe_queries_results.json", '../jsonData/recipesMap.json').createMap()

In [5]:
import loginOpenSearch as lop

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
    
client = login[0]
index_name = login[1]


----------------------------------------------------------------------------------- INDEX SETTINGS
{'user209': {'settings': {'index': {'creation_date': '1648397529683',
                                    'knn': 'true',
                                    'number_of_replicas': '0',
                                    'number_of_shards': '4',
                                    'provided_name': 'user209',
                                    'refresh_interval': '1s',
                                    'uuid': '41z7d69ETNyxdH2mDS7Vew',
                                    'version': {'created': '135238227'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'user209': {'mappings': {'properties': {'description': {'similarity': 'BM25',
                                                         'type': 'text'},
                                         'recipeDescription': {'fields': {'keyword': {'ignore_above': 256,
                       

In [6]:
import createIndexes as ci

ci.CreateIndexes(client, index_name).createIndexStructure()

In [ ]:
import computeEmbeddings as ce

ce.CreateEmbeddingsMap("../jsonData/recipesMap.json","../jsonData/recipesEmbeddingsMap.json").createMap()

In [7]:
import indexRecipes as ir

ir.IndexRecipes("../jsonData/recipesMap.json",client, index_name).indexRecipes()


updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated


In [8]:
import textBasedSearch as tbs

tbs.TextBasedSearch(client, index_name).queryOpenSearch('how to make chicken')


QUERY: chicken

Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': 'amzn1.alexa.kitchen.document.wholefoods.pdid.NTM3MGVhNzVjNjY5MjkxZTIyNTcwYTVl',
                    '_index': 'user209',
                    '_score': 3.7398136,
                    '_source': {},
                    '_type': '_doc',
                    'fields': {'recipeDescription': ['perfect family different '
                                                     'taste picky eater person '
                                                     'make dinner exactly way '
                                                     'want prepared rotisserie '
                                                     'chicken make add chicken '
                                                     'recipe snap'],
                               'recipeId': ['amzn1.alexa.kitchen.document.wholefoods.pdid.NTM3MGVhNzVjNjY5MjkxZTIyNTcwYTVl'],
                               'reci